# Naive Bayes Klasifikasi Gaussian pada Data Iris

**Nama  : Shohibun Naufal**

**NIM   : 210411100013**

**Kelas : Penambangan Data-C**

Naive Bayes pada dasarnya adalah sebuah algoritma klasifikasi yang berdasarkan pada teorema Bayes dengan mengasumsikan bahwa setiap fitur (atau atribut) pada data yang diamati adalah independen satu sama lain. Oleh karena itu, algoritma ini dianggap "naif" karena tidak mempertimbangkan interaksi antara fitur.

Dalam klasifikasi, Naive Bayes menghitung probabilitas setiap kelas yang mungkin berdasarkan pada nilai fitur yang diamati, dan kemudian memilih kelas dengan probabilitas tertinggi sebagai prediksi. Algoritma ini sering digunakan dalam analisis teks, seperti pengklasifikasian email spam atau klasifikasi dokumen berdasarkan topik, tetapi juga dapat diterapkan pada berbagai jenis masalah klasifikasi lainnya.

1. import library pandas, numpy, dan sklearn yang diperlukan

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

2. import data kita

In [3]:
df = pd.read_csv('https://gist.githubusercontent.com/shohibunnaufal/d3516ed009a4ab1dcc5c2fa2a378d02c/raw/2140f62dfd210658868e79e56a18c66a381520f2/IrisDataSet')

jumlah_baris = df.shape[0]
df.head(jumlah_baris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,NaN,3.5,1.4,0.2,setosa
1,?,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


3. Mengecek nilai yang hilang/kosong pada data

In [4]:
df_copy = df.copy()  #duplikat data asli
df_copy.isnull().sum()

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

4. Ganti nilai yang mengandung tanda tanya (?) menjadi nilai kosong

In [5]:
df_copy = df_copy.replace('?',np.nan)

5. Cek lagi berapa nilai yang kosong pada data

In [6]:
df_copy.isnull().sum()

sepal_length    2
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

6. Setelah itu kita lihat type datanya, jika type datanya bukan numerik/float maka perlu diganti type datanya

In [7]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  148 non-null    object 
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(3), object(2)
memory usage: 6.0+ KB


7. Mengubah semuanya tipe data pada fitur menjadi tipe data numerik

In [8]:
# mengubah fitur sepal_length menjadi tipe data float
df_copy['sepal_length'] = df_copy['sepal_length'].astype(float)

# mengubah fitur species menjadi tipe data int yang awalnya categorical
le = LabelEncoder()
df_copy['species'] = le.fit_transform(df_copy['species'])

8. Cek kembali tipe data pada semua fitur

In [9]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  148 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


9. Mengisi nilai yang hilang/kosong. Disini kita menggunakan mean, mean tidak selalu menjadi nilai yang baik untuk mengisi nilai yang hilang. Kita juga bisa menggunakan rumus untuk mengisinya.

In [10]:
df_copy = df_copy.fillna(df_copy.mean(numeric_only=True))

10. Cek apakah masih ada data yang kosong

In [11]:
df_copy.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

11. Sekarang coba kita tampilkan data yang terbaru

In [12]:
df_copy.head(jumlah_baris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.85473,3.5,1.4,0.2,0
1,5.85473,3.0,1.4,0.2,0
2,4.70000,3.2,1.3,0.2,0
3,4.60000,3.1,1.5,0.2,0
4,5.00000,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.70000,3.0,5.2,2.3,2
146,6.30000,2.5,5.0,1.9,2
147,6.50000,3.0,5.2,2.0,2
148,6.20000,3.4,5.4,2.3,2


12. Melakukan normalisasi data, yaitu mengubah setiap nilai ke dalam rentang antara 0 hingga 1. Kita akan menggunakan metode min-max, yaitu: x - min / max - min. Terdapat juga rumus-rumus lain yang dapat digunakan, misalnya metode simple feature scaling atau metode z-score. Namun perlu diingat bahwa kolom target tidak akan dinormalisasi (kolom species).

In [13]:
species_column = df_copy['species']
df_copy = df_copy[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].copy()
df_copy = (df_copy - df_copy.min())/(df_copy.max() - df_copy.min())
df_copy['species'] = species_column.copy()
df_copy.head(jumlah_baris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,0.431869,0.625000,0.067797,0.041667,0
1,0.431869,0.416667,0.067797,0.041667,0
2,0.111111,0.500000,0.050847,0.041667,0
3,0.083333,0.458333,0.084746,0.041667,0
4,0.194444,0.666667,0.067797,0.041667,0
...,...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667,2
146,0.555556,0.208333,0.677966,0.750000,2
147,0.611111,0.416667,0.711864,0.791667,2
148,0.527778,0.583333,0.745763,0.916667,2


13. Pemotongan Data atau Data Slicing

> pisahkan data menjadi data training and data test. Kita dapat dengan mudah melakukan langkah ini menggunakan metode train_test_split() dari sklearn:

> Variabel features_train berisi data fitur untuk training, features_test berisi data fitur untuk test, target_train berisi target untuk training, dan target_test berisi target untuk test.

In [14]:
features = df_copy.values[:,:4]  #Variabel features diisi dengan nilai features dari dataset adult_df_rev dengan slicing [:,:4] artinya mengambil seluruh baris dan kolom 0 hingga kolom 3 (kolom ke-4 tidak diambil karena merupakan variabel target).
target = df_copy.values[:,4]  #Variabel target diisi dengan nilai target dari dataset adult_df_rev dengan slicing [:,4] artinya mengambil seluruh baris dan hanya kolom ke-4 (variabel target).
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size = 0.33, random_state = 10)  #Kemudian menggunakan fungsi train_test_split, dataset yang sudah dipisahkan dijadikan argumen, dengan tambahan argumen test_size = 0.33 yang berarti dataset akan dibagi menjadi 33% data uji dan 67% data latih, dan random_state = 10 sebagai seed untuk menghasilkan hasil yang sama setiap kali dijalankan.

14. Implementasi Gaussian Naive Bayes

> Setelah selesai preprocessing data. saatnya menerapkan algoritma pembelajaran mesin atau machine learning. Kita akan menggunakan modul GaussianNB sklearn:

In [15]:
clf = GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

15. Akurasi model Gaussian Naive Bayes

In [16]:
accuracy = accuracy_score(target_test, target_pred, normalize = True)

print(f"Akurasi model Naive Bayes: {accuracy:.2%}")

Akurasi model Naive Bayes: 94.00%


## Kesimpulan

Berdasarkan nilai diatas tingkat akurasinya 94% . Ini menunjukkan data kita Bagus. Kita dapat membuat kumpulan data uji acak dan menguji model untuk mengetahui seberapa baik kinerja model Gaussian Naive Bayes yang terlatih.